# **Practice with eager execution **

In [30]:
import pandas as pd
import numpy as np

In [ ]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

## Eager Execution

- Compute with concrete values
- Doesn't break the flow of computing gradients

In [35]:
x = [[2.]]
m = tf.matmul(x,x)
print("{}".format(m), type(m))

[[ 4.]] <class 'EagerTensor'>


In [36]:
x = tf.constant([[1,2],[3,4]])
x

<tf.Tensor: id=528474, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [37]:
x = x.numpy()
x

array([[1, 2],
       [3, 4]], dtype=int32)

In [38]:
np.multiply(x,x)

array([[ 1,  4],
       [ 9, 16]], dtype=int32)

##  Dataset API

- Build complex input pipelines
- Two new abstractions
        - tf.data.Dataset
        - tf.data.Iterator

In [73]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
test_dataset_url = "http://download.tensorflow.org/data/iris_test.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)
test_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url),
                                           origin=train_dataset_url)
print("Local copy of the dataset file: {}".format(train_dataset_fp))
print("Local copy of the dataset file: {}".format(test_dataset_fp))

Local copy of the dataset file: /Users/shiwang/.keras/datasets/iris_training.csv
Local copy of the dataset file: /Users/shiwang/.keras/datasets/iris_test.csv


In [74]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [75]:
def parse_csv(line):
    example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
    parsed_line = tf.decode_csv(line, example_defaults)
    
    # First 4 fields are features, combine into single tensor
    features = tf.reshape(parsed_line[:-1], shape=(4,))
    
    # Last field is the label
    label = tf.reshape(parsed_line[-1], shape=())
    return features, label

Create a pipeline

In [76]:
train_dataset = tf.data.TextLineDataset(train_dataset_fp).skip(1).map(parse_csv)

In [77]:
features, label = iter(train_dataset).next()
features

<tf.Tensor: id=1056979, shape=(4,), dtype=float32, numpy=array([ 6.4000001 ,  2.79999995,  5.5999999 ,  2.20000005], dtype=float32)>

In [58]:
label

<tf.Tensor: id=1039640, shape=(), dtype=int32, numpy=2>

In [59]:
train_dataset = train_dataset.shuffle(buffer_size=1000).repeat(2).batch(32)

In [60]:
features, label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor([ 5.19999981  3.4000001   1.39999998  0.2       ], shape=(4,), dtype=float32)
example label: tf.Tensor(0, shape=(), dtype=int32)


## Keras API

- High level neural network APU
- Run on top of Tensorflow, CNTK, Theano

In [46]:
model = tf.keras.Sequential()
layer1 = tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)) # input shape required
layer2 = tf.keras.layers.Dense(10, activation="relu")
layer3 = tf.keras.layers.Dense(3)
model.add(layer1)
model.add(layer2)
model.add(layer3)

- Cannot use the following with Eager Execution

In [47]:
#model.compile(loss='categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])
#
#model.fit(features, labels, nb_epoch=10, verbose=1)

- Only native loss and optimizer is compatible with eager execution
- The grad function uses the loss function and the tf.GradientTape to record operations that compute the gradients used to optimize our model.

In [48]:
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape: # all forward-pass operations get recorded to a "tape
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

## Start Train Loop

In [49]:
num_epochs = 400

In [50]:
# keep results for plotting
train_loss_results = []
train_accuracy_results = []

for epoch in range(num_epochs):
    # For each epoch, run over Entire dataset
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()
    
    # For batch size of 32
    for x, y in train_dataset:  # return a batch at every call of train_dataset
        
        # Call our gradient function, and Optimize for model.variables
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables), 
                                 global_step = tf.train.get_or_create_global_step())
        
        # Track progress
        ### Print out value at Run Time
        epoch_loss_avg(loss(model, x, y))
        epoch_accuracy(tf.argmax(model(x), axis = 1, output_type=tf.int32), 
                       y)
        
    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.028, Accuracy: 59.583%
Epoch 050: Loss: 0.226, Accuracy: 95.833%
Epoch 100: Loss: 0.117, Accuracy: 98.333%
Epoch 150: Loss: 0.095, Accuracy: 97.917%
Epoch 200: Loss: 0.070, Accuracy: 98.333%
Epoch 250: Loss: 0.077, Accuracy: 98.750%
Epoch 300: Loss: 0.067, Accuracy: 98.333%
Epoch 350: Loss: 0.062, Accuracy: 97.917%


Look at some of the Eager Execution outputs

In [51]:
loss(model, x, y)

<tf.Tensor: id=1039487, shape=(), dtype=float32, numpy=0.033696391>

In [52]:
grad(model, x, y)

[<tf.Tensor: id=1039575, shape=(4, 10), dtype=float32, numpy=
 array([[ 0.        , -0.02939828,  0.        ,  0.        ,  0.        ,
         -0.01705971,  0.        , -0.06066043,  0.06475139,  0.        ],
        [ 0.        , -0.00836889,  0.        ,  0.        ,  0.        ,
         -0.00403176,  0.        , -0.01694808,  0.01688088,  0.        ],
        [ 0.        , -0.02166644,  0.        ,  0.        ,  0.        ,
         -0.01374358,  0.        , -0.04480889,  0.04893772,  0.        ],
        [ 0.        , -0.01786771,  0.        ,  0.        ,  0.        ,
         -0.01130618,  0.        , -0.03771346,  0.04218885,  0.        ]], dtype=float32)>,
 <tf.Tensor: id=1039573, shape=(10,), dtype=float32, numpy=
 array([ 0.        , -0.00164259,  0.        ,  0.        ,  0.        ,
        -0.00072838,  0.        , -0.00316354,  0.00282967,  0.        ], dtype=float32)>,
 <tf.Tensor: id=1039571, shape=(10, 10), dtype=float32, numpy=
 array([[ 0.        ,  0.        ,  0

Predict with model

In [53]:
class_ids = ["Iris setosa", "Iris versicolor", "Iris virginica"]

predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    name = class_ids[class_idx]
    print("Example {} prediction: {}".format(i, name))

Example 0 prediction: Iris setosa
Example 1 prediction: Iris versicolor
Example 2 prediction: Iris virginica


# Revist Eager Execution with a simple example

Revist Gradient Tape

In [61]:
w = tfe.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, [w])
print(grad) 

[<tf.Tensor: id=1039677, shape=(1, 1), dtype=float32, numpy=array([[ 2.]], dtype=float32)>]


Customized Model

In [62]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tfe.Variable(5., name = 'W')
        self.b = tfe.Variable(10., name = 'b')
    
    def predict(self, inputs):
        return inputs * self.W + self.b

Create sample inputs

In [63]:
training_inputs = tf.random_normal([1000])
noise = tf.random_normal([1000])
training_outputs = training_inputs * 3 + 2 + noise
training_outputs[0]

<tf.Tensor: id=1039693, shape=(), dtype=float32, numpy=4.7433739>

Define loss

In [64]:
def loss(model, x, y):
    error = model.predict(x) - y
    return tf.reduce_mean(tf.square(error))

Define Gradients

In [65]:
def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss_ = loss(model, x, y)
    return tape.gradient(loss_, [model.W, model.b])

Define Optimizer and Model

In [66]:
model = Model()
opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

Initial loss: 71.751


Start Train Loop

In [67]:
for i in range(500):
    grads = grad(model, training_inputs, training_outputs)
    opt.apply_gradients(zip(grads, [model.W, model.b]), 
                        global_step=tf.train.get_or_create_global_step())
    
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.b.numpy()))  

Loss at step 000: 68.835
Loss at step 020: 30.275
Loss at step 040: 13.659
Loss at step 060: 6.478
Loss at step 080: 3.364
Loss at step 100: 2.010
Loss at step 120: 1.419
Loss at step 140: 1.161
Loss at step 160: 1.047
Loss at step 180: 0.997
Loss at step 200: 0.974
Loss at step 220: 0.965
Loss at step 240: 0.960
Loss at step 260: 0.958
Loss at step 280: 0.957
Loss at step 300: 0.957
Loss at step 320: 0.957
Loss at step 340: 0.957
Loss at step 360: 0.957
Loss at step 380: 0.957
Loss at step 400: 0.957
Loss at step 420: 0.957
Loss at step 440: 0.957
Loss at step 460: 0.957
Loss at step 480: 0.957
Final loss: 0.957
W = 2.9841554164886475, B = 1.983044147491455


# Practice with Estimator API

<img src="https://www.tensorflow.org/images/tensorflow_programming_environment.png" width="600">

In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


Import data as Pandas Dataframe

In [21]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

def load_data(label_name='Species'):
    # Just copies a remote CSV file to a local file system
    train_path = tf.keras.utils.get_file(fname=TRAIN_URL.split('/')[-1],
                                         origin=TRAIN_URL)
    train = pd.read_csv(filepath_or_buffer=train_path,
                        names=CSV_COLUMN_NAMES,  # list of column names
                        header=0  # ignore the first row of the CSV file.
                       )
    train_features, train_label = train, train.pop(label_name)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_features, test_label = test, test.pop(label_name)
    return (train_features, train_label), (test_features, test_label)

(train_feature, train_label), (test_feature, test_label) = load_data()

In [22]:
train_feature.shape # Just pandas DF

(120, 4)

In [23]:
train_label.shape

(120,)

In [24]:
test_feature.head(3)

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5


Define `Feature Column`

<img src="https://www.tensorflow.org/images/feature_columns/inputs_to_model_bridge.jpg" width="800">

In [25]:
# Interpret the data in each feature as its literal floating-point value
my_feature_columns = [
    tf.feature_column.numeric_column(key='SepalLength',dtype=tf.float64),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

*Other examples of feature_column for categorical columns*

<img src="https://www.tensorflow.org/images/feature_columns/categorical_column_with_identity.jpg" width="300">

In [77]:
# https://www.tensorflow.org/get_started/feature_columns
# Feature B is categorical
identity_feature_column = tf.feature_column.categorical_column_with_identity(
    key='my_feature_b',
    num_buckets=4) # Values [0, 4)

def input_fn():
    ...
    return ({ 'my_feature_a':[7, 9, 5, 2], 'my_feature_b':[3, 1, 2, 2] },
            [Label_values])

# Feature B is a word
vocabulary_feature_column = tf.feature_column.categorical_column_with_vocabulary_file(
        key='my_feature_b',
        #vocabulary_list=["kitchenware", "electronics", "sports"]
        vocabulary_file="product_class.txt",
        vocabulary_size=3)

Define `Classifier`: Just use DNN Classifier

In [26]:
# DNNClassifer is a pre-made Estimator 
# https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10], # 2 hidden layers
    optimizer='Adagrad',
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmp53pto_yh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x18144d6550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Define `Input` Function

In [60]:
def train_input_fn(features, labels, batch_size): # Inputs are pandas df and series
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels)) # Using Dataset API
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next() # return a batch

- Train_feature is a Python dictionary in which:
    - Each key is the name of a feature.
    - Each value is an array containing the values for each example in the training set.

- Train_label is an array containing the values of the label for every example in the training set

In [59]:
# Convert into dictionary
dict(train_feature)['PetalLength'].shape

(120,)

In [58]:
# Output format
train_input_fn(train_feature, train_label, 30)

({'PetalLength': <tf.Tensor 'IteratorGetNext_1:0' shape=(?,) dtype=float64>,
  'PetalWidth': <tf.Tensor 'IteratorGetNext_1:1' shape=(?,) dtype=float64>,
  'SepalLength': <tf.Tensor 'IteratorGetNext_1:2' shape=(?,) dtype=float64>,
  'SepalWidth': <tf.Tensor 'IteratorGetNext_1:3' shape=(?,) dtype=float64>},
 <tf.Tensor 'IteratorGetNext_1:4' shape=(?,) dtype=int64>)

In [65]:
# Simple example to replace the batch above:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

In [45]:
def eval_input_fn(features, labels=None, batch_size=None):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

Start Training

- A lambda to capture the arguments while providing an input function that takes no arguments, as expected by the Estimator. 
- The steps argument tells the method to stop training after a number of training steps.

In [70]:
batch_size = 30
train_steps = 1000

classifier.train(
    input_fn=lambda:train_input_fn(train_feature, train_label, batch_size),
    steps=train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmpo55yo6y_/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3001 into /var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmpo55yo6y_/model.ckpt.
INFO:tensorflow:loss = 0.00118172, step = 3001
INFO:tensorflow:global_step/sec: 933.322
INFO:tensorflow:loss = 0.00113752, step = 3101 (0.108 sec)
INFO:tensorflow:global_step/sec: 2325.74
INFO:tensorflow:loss = 0.0010969, step = 3201 (0.043 sec)
INFO:tensorflow:global_step/sec: 2116.19
INFO:tensorflow:loss = 0.00105913, step = 3301 (0.047 sec)
INFO:tensorflow:global_step/sec: 2331.54
INFO:tensorflow:loss = 0.00102434, step = 3401 (0.043 sec)
INFO:tensorflow:global_step/sec: 2365.12
INFO:tensorflow:loss = 0.000991813,

Evaluate Model with test Data

In [62]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_feature, test_label, batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-26-22:34:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmpo55yo6y_/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-26-22:34:10
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0541685, global_step = 2000, loss = 1.62506

Test set accuracy: 0.967



Next Steps: Predictions

# Revisit Dataset API

In [92]:
type(train_feature)

pandas.core.frame.DataFrame

In [93]:
type(train_label)

pandas.core.series.Series

In [94]:
dataset = tf.data.Dataset.from_tensor_slices((dict(train_feature), train_label)) # Using Dataset API

In [95]:
dataset = dataset.shuffle(buffer_size=1000).repeat(count=None)
print(dataset)

<RepeatDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


In [96]:
#Note that the dataset has an unknown batch size because the last batch will have fewer elements.
print(dataset.batch(10))

<BatchDataset shapes: ({SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, (?,)), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


In [97]:
# At this point the Dataset contains (features_dict, labels) pairs.
# return dataset

# Customized Estimators

### Define Model Function

In [105]:
def my_model_fn(
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # An instance of tf.estimator.ModeKeys
   params):  # Additional configuration
    
    ### 1. Define Input, Hidden Layer, Output
    
    # Use `input_layer` to apply the feature columns.
    net = tf.feature_column.input_layer(features, params['feature_columns']) # Numeric, etc
    
    # Build the hidden layers, sized according to the 'hidden_units' param.
    for units in params['hidden_units']: # [10,10]
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
        
    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None) # n_class = 3
    
    # Compute Other metrics: Loss, Pred, Acc
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    predicted_classes = tf.argmax(logits, 1)
    accuracy = tf.metrics.accuracy(labels=labels,predictions=predicted_classes,name='acc_op')
    
    ### 2. Define Prediction
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis], #[0,1,2,3]
            'probabilities': tf.nn.softmax(logits), ## Softmax
            'logits': logits #[-,+]
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions) #Notice Return Type


    ### 3. Define Evaluation
    metrics = {'accuracy': accuracy}
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)


    ### 4. Define Train
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


### Define Classifier

In [113]:
classifier = tf.estimator.Estimator(
    model_fn=my_model_fn,
    params={
        'feature_columns': my_feature_columns,
        'hidden_units': [10, 10],
        'n_classes': 3,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmp0brj60st', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1814ba0d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [114]:
# Already Defined
#train_feature, train_label
#def train_input_fn:
#def eval_input_fn:
#my_feature_columns = 

### Train and Evaluate the Model

In [111]:
classifier.train(
    input_fn=lambda:train_input_fn(train_feature, train_label, batch_size = 32),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmpxkfol9hw/model.ckpt.
INFO:tensorflow:loss = 3.24971, step = 1
INFO:tensorflow:global_step/sec: 868.228
INFO:tensorflow:loss = 0.185193, step = 101 (0.117 sec)
INFO:tensorflow:global_step/sec: 1557.07
INFO:tensorflow:loss = 0.133819, step = 201 (0.064 sec)
INFO:tensorflow:global_step/sec: 1190.8
INFO:tensorflow:loss = 0.0371454, step = 301 (0.084 sec)
INFO:tensorflow:global_step/sec: 1208.18
INFO:tensorflow:loss = 0.0974107, step = 401 (0.083 sec)
INFO:tensorflow:global_step/sec: 1142.23
INFO:tensorflow:loss = 0.0538875, step = 501 (0.087 sec)
INFO:tensorflow:global_step/sec: 1146.14
INFO:tensorflow:loss = 0.125988, step = 601 (0.087 sec)
INFO:tensorflow:glo

In [112]:
classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_feature, test_label, batch_size = 32),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-26-23:28:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/s2/z1q3239559s96vnyx0npgcqn4bw6wc/T/tmpxkfol9hw/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-26-23:28:57
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.933333, global_step = 1000, loss = 0.0585234


{'accuracy': 0.93333334, 'global_step': 1000, 'loss': 0.058523417}